In [283]:
#@title Carrega Bibliotecas

# Carrega Pacotes
import requests
from bs4 import BeautifulSoup
import numpy as np
import numpy as np
import pandas as pd
import datetime
import random
from unidecode import unidecode
from sqlalchemy import create_engine 


In [284]:
#@title Scrap Sobrenomes

# Define Url de Sobrenomes
url = 'https://www.procob.com/os-sobrenomes-mais-comuns-do-brasil/'
 
# Conecta com URL
reqs = requests.get(url)
 
# Extrai informações de lista ordenada
soup = BeautifulSoup(reqs.text, 'lxml')
 
# Define Sobrenomes
Sobrenomes = np.random.choice([tag.text for tag in soup.find_all("ol")][0].split(), 90)
 
# Imprime primeiros Sobrenomes 
print(Sobrenomes[1:10])

['Cavalcanti' 'Campos' 'Costa' 'Silva' 'Oliveira' 'Souza' 'Nascimento'
 'Freitas' 'Ribeiro']


In [285]:
#@title Scrap Nome de Homens

# Define Url de Homens
url2 = 'https://www.minhavida.com.br/familia/materias/35919-100-nomes-para-meninos-mais-comuns-confira-lista'

# Conecta com URL
reqs = requests.get(url2)

# Extrai informações de lista ordenada
soup = BeautifulSoup(reqs.text, 'lxml')

# Define Sobrenomes
Homens = [tag.text for tag in soup.find_all("ul")][1].split()

# Imprime nomes de Homens
print(Homens[0:10])


['Miguel', 'Arthur', 'Heitor', 'Bernardo', 'Théo', 'Davi', 'Gabriel', 'Pedro', 'Samuel', 'Lorenzo']


In [286]:
#@title Scrap Nomes de Mulheres

# Define Url de Mulheres
url3 = 'https://www.minhavida.com.br/familia/materias/35925-100-nomes-para-meninas-mais-comuns-confira-lista'

# Conecta com URL
reqs = requests.get(url3)

# Extrai informações de lista ordenada
soup = BeautifulSoup(reqs.text, 'lxml')

# Define Sobrenomes
Mulheres = [tag.text for tag in soup.find_all("ul")][1].split()

print(Mulheres[1:10])

['Alice', 'Laura', 'Manuela', 'Valentina', 'Sophia', 'Isabella', 'Heloísa', 'Luiza', 'Júlia']


In [287]:
#@title Scrap Nomes de Produtos

# Define Url de Melhores 10 celulares
url4 = 'https://mobizoo.com.br/opiniao/celulares-mais-vendidos/'

# Conecta com URL
reqs = requests.get(url4)

# Extrai informações da url
soup = BeautifulSoup(reqs.text, 'lxml')

# Define Produtos de Celulares
Produtos_Celulares = [tag.text for tag in soup.find_all("th")][5::4]

# Imprime Produtos de Celulares
print(Produtos_Celulares)

# Define Url de Melhores Desktops
url5 = 'https://spy.com/articles/gadgets/electronics/best-desktop-computers-reviews-247841/'

# Conecta com URL
reqs = requests.get(url5)

# Extrai informações da url
soup = BeautifulSoup(reqs.text, 'lxml')

# Define Variavel auxiliar com titulos completos com categoria
Produtos_Desktop = [tag.text for tag in soup.find_all("h2")][:11]

# Define Produtos Desktops 
Produtos_Desktop = list(map(lambda w: w.split(". ", 1)[1], Produtos_Desktop))

# Concatena Produtos
Produtos = Produtos_Celulares + Produtos_Desktop

# Imprime Produtos
print(Produtos)


['iPhone 11', 'Galaxy A51', 'Redmi Note 8', 'Redmi Note 8 Pro', 'iPhone SE 2', 'iPhone XR', 'iPhone 11 Pro Max', 'Redmi 8A', 'Redmi 8', 'iPhone 11 Pro']
['iPhone 11', 'Galaxy A51', 'Redmi Note 8', 'Redmi Note 8 Pro', 'iPhone SE 2', 'iPhone XR', 'iPhone 11 Pro Max', 'Redmi 8A', 'Redmi 8', 'iPhone 11 Pro', 'Apple iMac with Retina 5K Display', 'HP Envy Desktop', 'Acer Aspire TC-895 Tower', 'Microsoft Surface Studio 2', 'HP Envy 32 All-In-One PC', 'HP Pavilion All-In-One', 'Mac Mini', 'Intel NUC 10 Performance Mini PC NUC10i5FNHCA', 'Alienware Aurora R10 Ryzen Edition R10', 'CyberpowerPC Gamer Xtreme VR', 'ASUS Chromebit CS10 Stick-Desktop PC']


In [288]:
#@title Gera emails e datas

# Conding
encoding = "utf-8"

# Declara email de Clientes
email_Cliente = [unidecode(i) + j for i, j in zip(list(Homens[0:89]), np.random.choice(["@gmail.com","@yahoo.com","@bh.com"], 90).tolist())] 

# Email de Fornecedor
email_Fornecedor = [unidecode(i) + j for i, j in zip(list(Homens[90:99]), np.random.choice(["@gmail.com","@yahoo.com","@bh.com"], 90).tolist())]

# Data de Inicio
inicio = datetime.date(2020, 1, 1)
fim = datetime.date(2020, 10, 1)

# Calcula Data
tempo_entre_datas = fim - inicio
dias_entre_datas = tempo_entre_datas.days
dias_aleatorios = random.randrange(dias_entre_datas)
data_aleatoria = inicio + datetime.timedelta(days=dias_aleatorios)

# Data Ordem
Data_Ordem = pd.date_range(start = '2020-01-01', end = '2020-10-01', periods=5000)

# Imprime data ordem
print(Data_Ordem[1:10])

# Data Entrega
Data_Entrega = [data + datetime.timedelta(days=random.randrange(dias_entre_datas)) for data in Data_Ordem]

# Imprime data entrega
print(Data_Entrega[1:10])

# Imprime data
print(email_Cliente)



DatetimeIndex(['2020-01-01 01:18:55.667133426',
               '2020-01-01 02:37:51.334266853',
               '2020-01-01 03:56:47.001400280',
               '2020-01-01 05:15:42.668533706',
               '2020-01-01 06:34:38.335667133',
               '2020-01-01 07:53:34.002800560',
               '2020-01-01 09:12:29.669933986',
               '2020-01-01 10:31:25.337067413',
               '2020-01-01 11:50:21.004200840'],
              dtype='datetime64[ns]', freq=None)
[Timestamp('2020-04-15 01:18:55.667133426'), Timestamp('2020-01-30 02:37:51.334266853'), Timestamp('2020-09-29 03:56:47.001400280'), Timestamp('2020-02-13 05:15:42.668533706'), Timestamp('2020-04-22 06:34:38.335667133'), Timestamp('2020-07-21 07:53:34.002800560'), Timestamp('2020-02-02 09:12:29.669933986'), Timestamp('2020-07-05 10:31:25.337067413'), Timestamp('2020-07-26 11:50:21.004200840')]
['Miguel@bh.com', 'Arthur@bh.com', 'Heitor@bh.com', 'Bernardo@bh.com', 'Theo@gmail.com', 'Davi@bh.com', 'Gabriel@yahoo.co

In [307]:
#@title Declara tabelas (dataframes)

# Declara data frame de Clientes com 500 clientes
Cliente = pd.DataFrame({'ID_Cliente' : range(89),
                       'Nome' : np.random.choice(Homens + Mulheres, 89).tolist(),
                       'Email' : email_Cliente})

# Imprime Cliente
print(Cliente.head())

# Declara data frame de Fornecedor com 50 Fornecedores
Fornecedor = pd.DataFrame({'ID_Fornecedor' : range(9),
                           'Nome' : Homens[90:99],
                           'Email' : email_Fornecedor})

# Imprime Fornecedor
print(Fornecedor.head())

# Declara data frame de Ordem com 5000 ordens
Ordem = pd.DataFrame({'ID_Ordem' : range(5000),
                      'ID_Cliente' : np.random.choice(range(500), 5000).tolist(),
                      'ID_Produto' : np.random.choice(range(20), 5000).tolist(),
                      'Data' : Data_Ordem})

# Remove Horario
Ordem['Data'] = [str(i) for i in pd.to_datetime(Ordem['Data']).dt.date]

# Imprime Ordem
print(Ordem.head())

# Declara dataframe de Produto com 10 produtos
Produto = pd.DataFrame({'ID_Produto' : range(21),
                        'ID_Fornecedor' : np.random.choice(range(9), 21).tolist(),
                        'Nome' : Produtos })

# Imprime Produto
print(Produto.head())

# Declara data frame de entrega com 5000 entregas
Entrega = pd.DataFrame({'ID_Entrega' : range(5000),
                        'ID_Produto' : Ordem.ID_Produto,
                        'Data' : Data_Entrega})

# Remove Horario
Entrega['Data'] = [str(i) for i in pd.to_datetime(Entrega['Data']).dt.date]

# Imprime Entrega
print(Entrega.head())

# Declara Data Frame Sobrenome
Sobrenome = pd.DataFrame({'ID_Cliente' : range(90),
                          'Sobrenome' : Sobrenomes})

# Imprime Sobrenome
print(Sobrenome.head())


   ID_Cliente      Nome            Email
0           0   Augusto    Miguel@bh.com
1           1     Elisa    Arthur@bh.com
2           2     Maria    Heitor@bh.com
3           3  Bernardo  Bernardo@bh.com
4           4      Luiz   Theo@gmail.com
   ID_Fornecedor     Nome              Email
0              0   Oliver   Oliver@gmail.com
1              1   Arthur   Arthur@gmail.com
2              2  Gabriel  Gabriel@yahoo.com
3              3     Ryan        Ryan@bh.com
4              4     Raul        Raul@bh.com
   ID_Ordem  ID_Cliente  ID_Produto        Data
0         0          72          16  2020-01-01
1         1          74          18  2020-01-01
2         2         423           6  2020-01-01
3         3         444           3  2020-01-01
4         4         329          15  2020-01-01
   ID_Produto  ID_Fornecedor              Nome
0           0              5         iPhone 11
1           1              0        Galaxy A51
2           2              8      Redmi Note 8
3       

In [290]:
#@title Transforma Cliente em sql

# Converte para SQL
rows = Cliente.to_records(index=False)
values = ', '.join(map(str, rows))
sql = "INSERT INTO Cliente VALUES {}".format(values)
print(sql.replace("""\'""", """\""""))
  
# Cria Motor de Busca 
engine = create_engine('sqlite:///ibdtp.db',  
                       echo = False) 

# Apaga tabela se já existir
engine.execute("DROP TABLE IF EXISTS Cliente;")
  
# Gera tabela 
engine.execute("CREATE TABLE Cliente ( \
                ID_Cliente mediumint(8) NOT NULL,\
                Nome varchar(255) default NULL,\
                Email varchar(255) default NULL,\
                PRIMARY KEY (ID_Cliente) \
                );")

# Popula tabela
engine.execute(str(sql))

# Executa query na tabela
Cliente_sql = pd.read_sql('SELECT * FROM Cliente LIMIT 5', 
				con = engine) 


print(Cliente_sql)

INSERT INTO Cliente VALUES (0, "Henrique", "Miguel@bh.com"), (1, "Heitor", "Arthur@bh.com"), (2, "Otávio", "Heitor@bh.com"), (3, "Enzo", "Bernardo@bh.com"), (4, "Pedro", "Theo@gmail.com"), (5, "Mathias", "Davi@bh.com"), (6, "Aurora", "Gabriel@yahoo.com"), (7, "Miguel", "Pedro@gmail.com"), (8, "Thomas", "Samuel@gmail.com"), (9, "Kauê", "Lorenzo@gmail.com"), (10, "Arthur", "Benjamin@bh.com"), (11, "Leonardo", "Matheus@bh.com"), (12, "Cecília", "Lucas@bh.com"), (13, "Lívia", "Benicio@bh.com"), (14, "Manuela", "Gael@yahoo.com"), (15, "Flor", "Joaquim@gmail.com"), (16, "Lívia", "Nicolas@yahoo.com"), (17, "Miguel", "Henrique@yahoo.com"), (18, "Joana", "Rafael@yahoo.com"), (19, "Kauê", "Isaac@bh.com"), (20, "Pedro", "Guilherme@bh.com"), (21, "Raul", "Murilo@gmail.com"), (22, "Maria", "Lucca@gmail.com"), (23, "Gabriel", "Gustavo@gmail.com"), (24, "Cauã", "Joao@bh.com"), (25, "Cecília", "Miguel@bh.com"), (26, "Maria", "Noah@bh.com"), (27, "Maria", "Felipe@bh.com"), (28, "Clarice", "Anthony@gmai

In [291]:
#@title Transforma Sobrenome em sql

# Converte para SQL
rows = Sobrenome.to_records(index=False)
values = ', '.join(map(str, rows))
sql = "INSERT INTO Sobrenome VALUES {}".format(values)
print(sql.replace("""\'""", """\""""))

# Apaga tabela se já existir
engine.execute("DROP TABLE IF EXISTS Sobrenome;")
  
# Gera tabela 
engine.execute("CREATE TABLE Sobrenome ( \
                ID_Cliente mediumint(8) NOT NULL, \
                Sobrenome varchar(255) default NULL, \
                PRIMARY KEY (ID_Cliente) \
                );")

# Popula tabela
engine.execute(str(sql))

# Executa query na tabela
Sobrenome_sql = pd.read_sql('SELECT * FROM Sobrenome LIMIT 5', 
				                    con = engine) 

# Imprime Sobrenome
print(Sobrenome_sql)

INSERT INTO Sobrenome VALUES (0, "Alves"), (1, "Cavalcanti"), (2, "Campos"), (3, "Costa"), (4, "Silva"), (5, "Oliveira"), (6, "Souza"), (7, "Nascimento"), (8, "Freitas"), (9, "Ribeiro"), (10, "Silva"), (11, "Rocha"), (12, "Pinto"), (13, "Rodrigues"), (14, "Pereira"), (15, "Alves"), (16, "Santos"), (17, "Barros"), (18, "Barbosa"), (19, "Santos"), (20, "Dias"), (21, "Barbosa"), (22, "Nascimento"), (23, "Nascimento"), (24, "Fernandes"), (25, "Santos"), (26, "Castro"), (27, "Monteiro"), (28, "Rocha"), (29, "Mendes"), (30, "Ribeiro"), (31, "Pinto"), (32, "Santos"), (33, "Nascimento"), (34, "Monteiro"), (35, "Mendes"), (36, "Santos"), (37, "Oliveira"), (38, "Barros"), (39, "Souza"), (40, "Martins"), (41, "Rodrigues"), (42, "Freitas"), (43, "Araújo"), (44, "Barbosa"), (45, "Barbosa"), (46, "Cardoso"), (47, "Araújo"), (48, "Oliveira"), (49, "Souza"), (50, "Fernandes"), (51, "Freitas"), (52, "Mendes"), (53, "Barbosa"), (54, "Santos"), (55, "Cardoso"), (56, "Barros"), (57, "Campos"), (58, "Almei

In [292]:
#@title Transforma Fornecedor em sql

# Converte para SQL
rows = Fornecedor.to_records(index=False)
values = ', '.join(map(str, rows))
sql = "INSERT INTO Fornecedor VALUES {}".format(values)
print(sql.replace("""\'""", """\""""))

# Apaga tabela se já existir
engine.execute("DROP TABLE IF EXISTS Fornecedor;")
  
# show the complete data 
engine.execute("CREATE TABLE Fornecedor ( \
                ID_Fornecedor  mediumint(8) NOT NULL, \
                Nome varchar(255) default NULL, \
                Email varchar(255) default NULL, \
                PRIMARY KEY (ID_Fornecedor) \
              );")

# Popula tabela de clientes
engine.execute(str(sql))

# Consulta tabela de fornecedores
Fornecedor_sql = pd.read_sql('SELECT * FROM Fornecedor LIMIT 5', 
				con = engine) 

# Imprime resultados da query
print(Fornecedor_sql)

INSERT INTO Fornecedor VALUES (0, "Oliver", "Oliver@gmail.com"), (1, "Arthur", "Arthur@gmail.com"), (2, "Gabriel", "Gabriel@yahoo.com"), (3, "Ryan", "Ryan@bh.com"), (4, "Raul", "Raul@bh.com"), (5, "Luan", "Luan@bh.com"), (6, "Tomás", "Tomas@gmail.com"), (7, "Mathias", "Mathias@gmail.com"), (8, "Davi", "Davi@yahoo.com")
   ID_Fornecedor     Nome              Email
0              0   Oliver   Oliver@gmail.com
1              1   Arthur   Arthur@gmail.com
2              2  Gabriel  Gabriel@yahoo.com
3              3     Ryan        Ryan@bh.com
4              4     Raul        Raul@bh.com


In [293]:
#@title Transforma Produto em sql

# Converte para SQL
rows = Produto.to_records(index=False)
values = ', '.join(map(str, rows))
sql = "INSERT INTO Produto VALUES {}".format(values)
print(sql.replace("""\'""", """\""""))

# Apaga tabela se já existir
engine.execute("DROP TABLE IF EXISTS Produto;")
  
# show the complete data 
engine.execute("CREATE TABLE Produto ( \
                ID_Produto mediumint(8) NOT NULL, \
                ID_Fornecedor mediumint, \
                Nome varchar(255) default NULL, \
                PRIMARY KEY (ID_Produto) \
              );")

# Popula tabela de clientes
engine.execute(str(sql))

# Consulta tabela de fornecedores
Produto_sql = pd.read_sql('SELECT * FROM Produto LIMIT 5', 
				con = engine) 

# Imprime resultados da query
print(Produto_sql)

INSERT INTO Produto VALUES (0, 4, "iPhone 11"), (1, 8, "Galaxy A51"), (2, 3, "Redmi Note 8"), (3, 0, "Redmi Note 8 Pro"), (4, 2, "iPhone SE 2"), (5, 3, "iPhone XR"), (6, 0, "iPhone 11 Pro Max"), (7, 1, "Redmi 8A"), (8, 4, "Redmi 8"), (9, 0, "iPhone 11 Pro"), (10, 3, "Apple iMac with Retina 5K Display"), (11, 4, "HP Envy Desktop"), (12, 3, "Acer Aspire TC-895 Tower"), (13, 4, "Microsoft Surface Studio 2"), (14, 1, "HP Envy 32 All-In-One PC"), (15, 6, "HP Pavilion All-In-One"), (16, 4, "Mac Mini"), (17, 2, "Intel NUC 10 Performance Mini PC NUC10i5FNHCA"), (18, 1, "Alienware Aurora R10 Ryzen Edition R10"), (19, 0, "CyberpowerPC Gamer Xtreme VR"), (20, 5, "ASUS Chromebit CS10 Stick-Desktop PC")
   ID_Produto  ID_Fornecedor              Nome
0           0              4         iPhone 11
1           1              8        Galaxy A51
2           2              3      Redmi Note 8
3           3              0  Redmi Note 8 Pro
4           4              2       iPhone SE 2


In [294]:
#@title Transforma Ordem em sql

# Converte para SQL
rows = Ordem.to_records(index=False)
values = ', '.join(map(str, rows))
sql = "INSERT INTO Ordem VALUES {}".format(values)
print(sql.replace("""\'""", """\""""))

# Apaga tabela se já existir
engine.execute("DROP TABLE IF EXISTS Ordem;")
  
# show the complete data 
engine.execute("CREATE TABLE Ordem ( \
                ID_Ordem  mediumint(8) NOT NULL, \
                ID_Cliente mediumint, \
                ID_Produto mediumint, \
                Data varchar(255) default NULL, \
                PRIMARY KEY (ID_Ordem) \
                );")

# Popula tabela de clientes
engine.execute(str(sql))

# Consulta tabela de fornecedores
Ordem_sql = pd.read_sql('SELECT * FROM Ordem LIMIT 5', 
				con = engine) 

# Imprime resultados da query
print(Ordem_sql)

INSERT INTO Ordem VALUES (0, 74, 18, "2020-01-01"), (1, 189, 13, "2020-01-01"), (2, 364, 11, "2020-01-01"), (3, 344, 1, "2020-01-01"), (4, 407, 3, "2020-01-01"), (5, 91, 3, "2020-01-01"), (6, 365, 7, "2020-01-01"), (7, 351, 8, "2020-01-01"), (8, 161, 13, "2020-01-01"), (9, 332, 14, "2020-01-01"), (10, 260, 12, "2020-01-01"), (11, 418, 18, "2020-01-01"), (12, 400, 9, "2020-01-01"), (13, 452, 11, "2020-01-01"), (14, 22, 9, "2020-01-01"), (15, 340, 6, "2020-01-01"), (16, 134, 5, "2020-01-01"), (17, 183, 11, "2020-01-01"), (18, 143, 7, "2020-01-01"), (19, 274, 17, "2020-01-02"), (20, 171, 8, "2020-01-02"), (21, 76, 8, "2020-01-02"), (22, 68, 0, "2020-01-02"), (23, 332, 17, "2020-01-02"), (24, 259, 12, "2020-01-02"), (25, 90, 11, "2020-01-02"), (26, 299, 18, "2020-01-02"), (27, 73, 1, "2020-01-02"), (28, 114, 17, "2020-01-02"), (29, 459, 3, "2020-01-02"), (30, 492, 8, "2020-01-02"), (31, 418, 8, "2020-01-02"), (32, 422, 10, "2020-01-02"), (33, 9, 15, "2020-01-02"), (34, 13, 18, "2020-01-02"

In [295]:
#@title Transforma Entrega em sql

# Converte para SQL
rows = Entrega.to_records(index=False)
values = ', '.join(map(str, rows))
sql = "INSERT INTO Entrega VALUES {}".format(values)
print(sql.replace("""\'""", """\""""))

# Apaga tabela se já existir
engine.execute("DROP TABLE IF EXISTS Entrega;")
  
# show the complete data 
engine.execute("CREATE TABLE Entrega ( \
                ID_Entrega mediumint(8) NOT NULL, \
                ID_Produto mediumint, \
                Data varchar(255), \
                PRIMARY KEY (ID_Entrega) \
                );")

# Popula tabela de clientes
engine.execute(str(sql))

# Consulta tabela de fornecedores
Entrega_sql = pd.read_sql('SELECT * FROM Entrega LIMIT 5', 
				con = engine) 

# Imprime resultados da query
print(Entrega_sql)

INSERT INTO Entrega VALUES (0, 18, "2020-05-26"), (1, 13, "2020-04-15"), (2, 11, "2020-01-30"), (3, 1, "2020-09-29"), (4, 3, "2020-02-13"), (5, 3, "2020-04-22"), (6, 7, "2020-07-21"), (7, 8, "2020-02-02"), (8, 13, "2020-07-05"), (9, 14, "2020-07-26"), (10, 12, "2020-03-09"), (11, 18, "2020-06-09"), (12, 9, "2020-02-25"), (13, 11, "2020-01-04"), (14, 9, "2020-02-13"), (15, 6, "2020-02-20"), (16, 5, "2020-06-25"), (17, 11, "2020-01-07"), (18, 7, "2020-05-02"), (19, 17, "2020-08-12"), (20, 8, "2020-01-05"), (21, 8, "2020-03-03"), (22, 0, "2020-03-02"), (23, 17, "2020-02-15"), (24, 12, "2020-02-28"), (25, 11, "2020-03-03"), (26, 18, "2020-01-27"), (27, 1, "2020-06-20"), (28, 17, "2020-09-01"), (29, 3, "2020-01-25"), (30, 8, "2020-07-12"), (31, 8, "2020-04-21"), (32, 10, "2020-03-02"), (33, 15, "2020-01-08"), (34, 18, "2020-06-17"), (35, 15, "2020-08-09"), (36, 13, "2020-04-02"), (37, 3, "2020-03-07"), (38, 0, "2020-05-27"), (39, 2, "2020-03-17"), (40, 19, "2020-08-05"), (41, 2, "2020-01-30

In [296]:
#@title Consulta 1) π(IDCliente,   Nome,   Email) (Cliente)

# Consulta para projetar todos os valores da Tabela Cliente
%timeit Consulta1 = pd.read_sql('SELECT * FROM Cliente', con = engine)

# Consulta para projetar todos os valores da Tabela Cliente
Consulta1 = pd.read_sql('SELECT * FROM Cliente', con = engine)

# Imprime resultados da query
print(Consulta1)


1000 loops, best of 3: 1.46 ms per loop
    ID_Cliente      Nome              Email
0            0  Henrique      Miguel@bh.com
1            1    Heitor      Arthur@bh.com
2            2    Otávio      Heitor@bh.com
3            3      Enzo    Bernardo@bh.com
4            4     Pedro     Theo@gmail.com
..         ...       ...                ...
84          84   Beatriz        Luiz@bh.com
85          85   Gabriel      Felipe@bh.com
86          86   Beatriz  Anthony@yahoo.com
87          87     Maria     Gabriel@bh.com
88          88    Felipe   Martin@yahoo.com

[89 rows x 3 columns]


In [297]:
#@title Consulta 2) π_(IDProduto,   IDFornecedor,   Nome) (Produto)

# Avalia tempo gasto na consulta
%timeit Consulta2 = pd.read_sql('SELECT * FROM Produto', con = engine) 

# Consulta para projetar todos os valores da Tabela Produto
Consulta2 = pd.read_sql('SELECT * FROM Produto', con = engine)

# Imprime resultados da query
print(Consulta2)

The slowest run took 4.56 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 1.3 ms per loop
    ID_Produto  ID_Fornecedor                                           Nome
0            0              4                                      iPhone 11
1            1              8                                     Galaxy A51
2            2              3                                   Redmi Note 8
3            3              0                               Redmi Note 8 Pro
4            4              2                                    iPhone SE 2
5            5              3                                      iPhone XR
6            6              0                              iPhone 11 Pro Max
7            7              1                                       Redmi 8A
8            8              4                                        Redmi 8
9            9              0                                  iPhone 11 

In [298]:
#@title Consulta 3) Cliente  ⋈_(IDCliente=IDCliente)  Ordem

# Avalia tempo gasto na consulta
%timeit Consulta3 = pd.read_sql('SELECT c.ID_Cliente, c.Nome, c.Email, o.ID_Ordem, o.ID_Produto, o.Data FROM Cliente as c JOIN Ordem as o ON c.ID_Cliente = o.ID_Cliente', con = engine) 

# Consulta para projetar a jução de Cliente e Ordem
Consulta3 = pd.read_sql("SELECT c.ID_Cliente, c.Nome, c.Email, o.ID_Ordem, o.ID_Produto, o.Data \
                         FROM Cliente as c \
                         JOIN Ordem as o ON c.ID_Cliente = o.ID_Cliente",
                         con = engine)

# Imprime resultados da query
print(Consulta3)

100 loops, best of 3: 4.89 ms per loop
     ID_Cliente       Nome  ... ID_Produto        Data
0            74       Laís  ...         18  2020-01-01
1            22      Maria  ...          9  2020-01-01
2            76    Anthony  ...          8  2020-01-02
3            68   Benjamin  ...          0  2020-01-02
4            73    Joaquim  ...          1  2020-01-02
..          ...        ...  ...        ...         ...
894          42  Emanuelly  ...          8  2020-09-30
895           7     Miguel  ...          5  2020-09-30
896          28    Clarice  ...          9  2020-09-30
897          66    Gabriel  ...         19  2020-09-30
898          69  Emanuelly  ...          2  2020-09-30

[899 rows x 6 columns]


In [299]:
#@title Consulta 4) Produto  ⋈_(IDProduto=IDProduto)  Ordem

# Avalia tempo gasto na consulta
%timeit Consulta4 = pd.read_sql('SELECT p.ID_Produto, p.ID_Fornecedor, p.Nome as Nome_Produto, o.ID_Ordem, o.ID_Cliente, o.Data FROM Produto as p JOIN Ordem as o ON p.ID_Produto = o.ID_Produto', con = engine) 

# Com União de Conjuntos
%timeit Consulta4 = pd.read_sql('SELECT p.ID_Produto, p.ID_Fornecedor, p.Nome as Nome_Produto, o.ID_Ordem, o.ID_Cliente, o.Data FROM Produto as p, Ordem as o WHERE p.ID_Produto = o.ID_Produto', con = engine) 

# Consulta para projetar a jução de Produto e Ordem
Consulta4 = pd.read_sql("SELECT p.ID_Produto, p.ID_Fornecedor, p.Nome as Nome_Produto, o.ID_Ordem, o.ID_Cliente, o.Data \
                         FROM Produto as p \
                         JOIN Ordem as o ON p.ID_Produto = o.ID_Produto",
                         con = engine)

# Imprime resultados da query
print(Consulta4)

100 loops, best of 3: 16.9 ms per loop
100 loops, best of 3: 17 ms per loop
      ID_Produto  ID_Fornecedor  ... ID_Cliente        Data
0             18              1  ...         74  2020-01-01
1             13              4  ...        189  2020-01-01
2             11              4  ...        364  2020-01-01
3              1              8  ...        344  2020-01-01
4              3              0  ...        407  2020-01-01
...          ...            ...  ...        ...         ...
4995           7              1  ...        463  2020-09-30
4996          19              0  ...         66  2020-09-30
4997           6              0  ...        442  2020-09-30
4998           2              3  ...         69  2020-09-30
4999           7              1  ...        361  2020-10-01

[5000 rows x 6 columns]


In [300]:
#@title Consulta 5) Fornecedor  ⋈_(IDFornecedor=IDFornecedor)  Produto

# Avalia tempo gasto na consulta
%timeit Consulta5 = pd.read_sql('SELECT f.ID_Fornecedor, f.Nome, f.Email, p.ID_Produto, p.Nome as Nome_Produto  FROM Fornecedor as f JOIN Produto as p ON f.ID_Fornecedor = p.ID_Fornecedor', con = engine) 

# Com União de Conjuntos
%timeit Consulta5 = pd.read_sql('SELECT f.ID_Fornecedor, f.Nome, f.Email, p.ID_Produto, p.Nome as Nome_Produto  FROM Fornecedor as f, Produto as p WHERE f.ID_Fornecedor = p.ID_Fornecedor', con = engine) 

# Consulta para projetar a junção de Fornecedor e Produto
Consulta5 = pd.read_sql("SELECT f.ID_Fornecedor, f.Nome, f.Email, p.ID_Produto, p.Nome as Nome_Produto  \
                         FROM Fornecedor as f \
                         JOIN Produto as p ON f.ID_Fornecedor = p.ID_Fornecedor",
                         con = engine)

# Imprime resultados da query
print(Consulta5)

1000 loops, best of 3: 1.53 ms per loop
1000 loops, best of 3: 1.53 ms per loop
    ID_Fornecedor  ...                                   Nome_Produto
0               4  ...                                      iPhone 11
1               8  ...                                     Galaxy A51
2               3  ...                                   Redmi Note 8
3               0  ...                               Redmi Note 8 Pro
4               2  ...                                    iPhone SE 2
5               3  ...                                      iPhone XR
6               0  ...                              iPhone 11 Pro Max
7               1  ...                                       Redmi 8A
8               4  ...                                        Redmi 8
9               0  ...                                  iPhone 11 Pro
10              3  ...              Apple iMac with Retina 5K Display
11              4  ...                                HP Envy Desktop
12        

In [301]:
#@title Consulta 6) (Fornecedor  ⋈_(IDFornecedor=IDFornecedor)  Produto)⋈_(IDProduto=IDProduto)  Ordem

# Avalia tempo gasto na consulta com junção
%timeit Consulta6 = pd.read_sql("SELECT f.ID_Fornecedor, f.Nome, f.Email, p.ID_Produto, p.Nome as Nome_Produto, o.ID_Cliente, o.Data FROM Fornecedor as f JOIN Produto as p ON f.ID_Fornecedor = p.ID_Fornecedor JOIN Ordem as o ON p.ID_Produto = o.ID_Produto", con = engine) 

# Com União de Conjuntos
%timeit Consulta6 = pd.read_sql("SELECT f.ID_Fornecedor, f.Nome, f.Email, p.ID_Produto, p.Nome as Nome_Produto, o.ID_Cliente, o.Data FROM Fornecedor as f, Produto as p, Ordem as o WHERE f.ID_Fornecedor = p.ID_Fornecedor AND p.ID_Produto = o.ID_Produto", con = engine) 

# Consulta para projetar a junção de Fornecedor, Produto e Ordem
Consulta6 = pd.read_sql("SELECT f.ID_Fornecedor, f.Nome, f.Email, p.ID_Produto, p.Nome as Nome_Produto, o.ID_Cliente, o.Data \
                         FROM Fornecedor as f \
                         JOIN Produto as p ON f.ID_Fornecedor = p.ID_Fornecedor \
                         JOIN Ordem as o ON p.ID_Produto = o.ID_Produto",
                         con = engine)

# Imprime resultados da query
print(Consulta6)

100 loops, best of 3: 19.8 ms per loop
10 loops, best of 3: 19.2 ms per loop
      ID_Fornecedor    Nome  ... ID_Cliente        Data
0                 1  Arthur  ...         74  2020-01-01
1                 4    Raul  ...        189  2020-01-01
2                 4    Raul  ...        364  2020-01-01
3                 8    Davi  ...        344  2020-01-01
4                 0  Oliver  ...        407  2020-01-01
...             ...     ...  ...        ...         ...
4995              1  Arthur  ...        463  2020-09-30
4996              0  Oliver  ...         66  2020-09-30
4997              0  Oliver  ...        442  2020-09-30
4998              3    Ryan  ...         69  2020-09-30
4999              1  Arthur  ...        361  2020-10-01

[5000 rows x 7 columns]


In [302]:
#@title Consulta 7) (Produto  ⋈_(IDProduto=IDProduto)  Ordem)  ⋈_(IDCliente=IDCliente)  Cliente

# Avalia tempo gasto na consulta
%timeit Consulta7 = pd.read_sql("SELECT p.ID_Produto, p.ID_Fornecedor, c.Nome as Nome_Cliente, o.ID_Ordem, o.ID_Cliente, o.Data FROM Produto as p JOIN Ordem as o ON p.ID_Produto = o.ID_Produto JOIN Cliente as c ON o.ID_Cliente = c.ID_Cliente", con = engine) 

# Avalia Tempo gasto com consulta de união de conjuntos
%timeit Consulta7 = pd.read_sql("SELECT p.ID_Produto, p.ID_Fornecedor, c.Nome as Nome_Cliente, o.ID_Ordem, o.ID_Cliente, o.Data FROM Produto as p, Ordem as o, Cliente as c WHERE p.ID_Produto = o.ID_Produto AND o.ID_Cliente = c.ID_Cliente", con = engine) 

# Consulta para projetar a jução de Produto, Ordem e Cliente
Consulta7 = pd.read_sql("SELECT p.ID_Produto, p.ID_Fornecedor, c.Nome as Nome_Cliente, o.ID_Ordem, o.ID_Cliente, o.Data \
                         FROM Produto as p \
                         JOIN Ordem as o ON p.ID_Produto = o.ID_Produto \
                         JOIN Cliente as c ON o.ID_Cliente = c.ID_Cliente",
                         con = engine)

# Imprime resultados da query
print(Consulta7)

100 loops, best of 3: 5.29 ms per loop
100 loops, best of 3: 5.33 ms per loop
     ID_Produto  ID_Fornecedor Nome_Cliente  ID_Ordem  ID_Cliente        Data
0            18              1         Laís         0          74  2020-01-01
1             9              0        Maria        14          22  2020-01-01
2             8              4      Anthony        21          76  2020-01-02
3             0              4     Benjamin        22          68  2020-01-02
4             1              8      Joaquim        27          73  2020-01-02
..          ...            ...          ...       ...         ...         ...
894           8              4    Emanuelly      4983          42  2020-09-30
895           5              3       Miguel      4988           7  2020-09-30
896           9              0      Clarice      4991          28  2020-09-30
897          19              0      Gabriel      4996          66  2020-09-30
898           2              3    Emanuelly      4998          6

In [303]:
#@title Consulta 8) (Produto  ⋈_(IDProduto=IDProduto)  Ordem)  ⋈_(IDFornecedor=IDFornecedor)  Fornecedor

# Avalia tempo gasto na consulta com junção
%timeit Consulta8 = pd.read_sql("SELECT p.ID_Produto, p.ID_Fornecedor, f.Nome as Nome_Fornecedor, o.ID_Ordem, o.ID_Cliente, o.Data FROM Produto as p JOIN Ordem as o ON p.ID_Produto = o.ID_Produto JOIN Fornecedor as f ON p.ID_Fornecedor = f.ID_Fornecedor", con = engine) 

# Avalia tempo gasto na consulta com união de conjuntos
%timeit Consulta8 = pd.read_sql("SELECT p.ID_Produto, p.ID_Fornecedor, f.Nome as Nome_Fornecedor, o.ID_Ordem, o.ID_Cliente, o.Data FROM Produto as p, Ordem as o, Fornecedor as f WHERE p.ID_Produto = o.ID_Produto AND p.ID_Fornecedor = f.ID_Fornecedor", con = engine) 

# Consulta para projetar a jução de Produto, Ordem e Fornecedor
Consulta8 = pd.read_sql("SELECT p.ID_Produto, p.ID_Fornecedor, f.Nome as Nome_Fornecedor, o.ID_Ordem, o.ID_Cliente, o.Data \
                         FROM Produto as p \
                         JOIN Ordem as o ON p.ID_Produto = o.ID_Produto \
                         JOIN Fornecedor as f ON p.ID_Fornecedor = p.ID_Fornecedor",
                         con = engine)

# Imprime resultados da query
print(Consulta8)

100 loops, best of 3: 17.5 ms per loop
100 loops, best of 3: 18 ms per loop
       ID_Produto  ID_Fornecedor  ... ID_Cliente        Data
0              18              1  ...         74  2020-01-01
1              18              1  ...         74  2020-01-01
2              18              1  ...         74  2020-01-01
3              18              1  ...         74  2020-01-01
4              18              1  ...         74  2020-01-01
...           ...            ...  ...        ...         ...
44995           7              1  ...        361  2020-10-01
44996           7              1  ...        361  2020-10-01
44997           7              1  ...        361  2020-10-01
44998           7              1  ...        361  2020-10-01
44999           7              1  ...        361  2020-10-01

[45000 rows x 6 columns]


In [308]:
#@title Consulta 9) J_(COUNT IDCliente) (Cliente  ⋈_(IDCliente=IDCliente)  Ordem) 

# Avalia tempo gasto na consulta com junção
%timeit Consulta9 = pd.read_sql("SELECT c.ID_Cliente, c.Nome, COUNT(o.ID_Ordem) as Frequencia FROM Cliente as c JOIN Ordem as o ON c.ID_Cliente = o.ID_Cliente GROUP BY c.ID_Cliente ORDER BY Frequencia DESC", con = engine) 

# Avalia tempo gasto na consulta com união de conjuntos
%timeit Consulta9 = pd.read_sql("SELECT c.ID_Cliente, c.Nome, COUNT(o.ID_Ordem) as Frequencia FROM Cliente as c, Ordem as o WHERE c.ID_Cliente = o.ID_Cliente GROUP BY c.ID_Cliente ORDER BY Frequencia DESC", con = engine) 

# Consulta para calcular a quantidade de ordens por cliente
Consulta9 = pd.read_sql("SELECT c.ID_Cliente, c.Nome, COUNT(o.ID_Ordem) as Quantidade_de_Ordens  \
                         FROM Cliente as c \
                         JOIN Ordem as o ON c.ID_Cliente = o.ID_Cliente \
                         GROUP BY c.ID_Cliente \
                         ORDER BY Quantidade_de_Ordens DESC",
                         con = engine)

# Imprime resultados da query
print(Consulta9)

100 loops, best of 3: 2.56 ms per loop
100 loops, best of 3: 2.5 ms per loop
    ID_Cliente       Nome  Quantidade_de_Ordens
0           16      Lívia                    20
1           54     Sophia                    19
2           39    Heloísa                    17
3           31     Pérola                    16
4           42  Emanuelly                    16
..         ...        ...                   ...
84          55     Louise                     5
85          78      Maria                     5
86          86    Beatriz                     5
87          24       Cauã                     4
88          70    Cecília                     3

[89 rows x 3 columns]


In [305]:
#@title Consulta 10) J_(COUNT IDProduto) (Fornecedor  ⋈_(IDFornecedor=IDFornecedor)  Produto)

# Avalia tempo gasto na consulta com junção
%timeit Consulta10 = pd.read_sql("SELECT f.ID_Fornecedor, f.Nome, COUNT(p.ID_Produto) as Frequencia FROM Fornecedor as f JOIN Produto as p ON f.ID_Fornecedor = p.ID_Fornecedor GROUP BY f.ID_Fornecedor ORDER BY Frequencia DESC", con = engine) 

# Avalia tempo gasto na consulta com união de conjuntos
%timeit Consulta10 = pd.read_sql("SELECT f.ID_Fornecedor, f.Nome, COUNT(p.ID_Produto) as Frequencia FROM Fornecedor as f, Produto as p WHERE f.ID_Fornecedor = p.ID_Fornecedor GROUP BY f.ID_Fornecedor ORDER BY Frequencia DESC", con = engine) 

# Consulta para Calcular a quantidade de produtos por fornecedor
Consulta10 = pd.read_sql("SELECT f.ID_Fornecedor, f.Nome, COUNT(p.ID_Produto) as Quantidade_de_Produtos \
                         FROM Fornecedor as f \
                         JOIN Produto as p ON f.ID_Fornecedor = p.ID_Fornecedor \
                         GROUP BY f.ID_Fornecedor \
                         ORDER BY Quantidade_de_Produtos DESC",
                         con = engine)

# Imprime resultados da query
print(Consulta10)

1000 loops, best of 3: 1.34 ms per loop
1000 loops, best of 3: 1.36 ms per loop
   ID_Fornecedor     Nome  Quantidade_de_Produtos
0              4     Raul                       5
1              0   Oliver                       4
2              3     Ryan                       4
3              1   Arthur                       3
4              2  Gabriel                       2
5              5     Luan                       1
6              6    Tomás                       1
7              8     Davi                       1


In [306]:

# attach the data frame to the sql  
# with a name of the table  
# as "Employee_Data" 
Cliente.to_sql('Cliente',
               con = engine,  
               if_exists = 'append') 

# reading the sql database 
# with index "Names" 
df2 = pd.read_sql('Cliente', 
				con = engine) 
# show the dataframe 
print(df2) 

# print new line 
print() 

# show the type of df2 
print(type(df2)) 


OperationalError: ignored

In [ ]:
-- Gera Base de Dados
CREATE DATABASE dados;
USE dados;

-- Descarta tabela Cliente
DROP TABLE Cliente;

-- Gera Tabela Cliente
CREATE TABLE Cliente (
  ID_Cliente mediumint(8) unsigned NOT NULL auto_increment,
  Nome varchar(255) default NULL,
  Email varchar(255) default NULL,
  PRIMARY KEY (ID_Cliente)
) AUTO_INCREMENT=1;

-- Popula Tabela Cliente
INSERT INTO Cliente (ID_Cliente, Nome, Email) VALUES ("1","Miguel","Miguel@bh.com"), ("2","Arthur","Arthur@gmail.com"), ("3","Bernardo","Bernardo@bh.com"), ("4","Heitor","Heitor@bh.com"), ("5","Davi","Davi@yahoo.com"), ("6","Lorenzo","Lorenzo@yahoo.com"), ("7","Théo","Théo@yahoo.com"), ("8","Pedro","Pedro@yahoo.com"), ("9","Gabriel","Gabriel@yahoo.com"), ("10","Enzo","Enzo@gmail.com"), ("11","Matheus","Matheus@gmail.com"), ("12","Lucas","Lucas@gmail.com"), ("13","Benjamin","Benjamin@yahoo.com"), ("14","Nicolas","Nicolas@gmail.com"), ("15","Guilherme","Guilherme@yahoo.com"), ("16","Rafael","Rafael@gmail.com"), ("17","Joaquim","Joaquim@gmail.com"), ("18","Samuel","Samuel@yahoo.com"), ("19","Enzo Gabriel","Enzo Gabriel@gmail.com"), ("20","João Miguel","João Miguel@gmail.com"), ("21","Henrique","Henrique@yahoo.com"), ("22","Gustavo","Gustavo@gmail.com"), ("23","Murilo","Murilo@gmail.com"), ("24","Pedro Henrique","Pedro Henrique@bh.com"), ("25","Pietro","Pietro@gmail.com"), ("26","Lucca","Lucca@gmail.com"), ("27","Felipe","Felipe@yahoo.com"), ("28","João Pedro","João Pedro@gmail.com"), ("29","Isaac","Isaac@yahoo.com"), ("30","Benício","Benício@yahoo.com"), ("31","Daniel","Daniel@gmail.com"), ("32","Anthony","Anthony@gmail.com"), ("33","Leonardo","Leonardo@gmail.com"), ("34","Davi Lucca","Davi Lucca@yahoo.com"), ("35","Bryan","Bryan@bh.com"), ("36","Eduardo","Eduardo@bh.com"), ("37","João Lucas","João Lucas@yahoo.com"), ("38","Victor","Victor@bh.com"), ("39","João","João@yahoo.com"), ("40","Cauã","Cauã@bh.com"), ("41","Antônio","Antônio@yahoo.com"), ("42","Vicente","Vicente@yahoo.com"), ("43","Caleb","Caleb@bh.com"), ("44","Gael","Gael@yahoo.com"), ("45","Bento","Bento@gmail.com"), ("46","Caio","Caio@bh.com"), ("47","Emanuel","Emanuel@yahoo.com"), ("48","Vinícius","Vinícius@bh.com"), ("49","João Guilherme","João Guilherme@bh.com"), ("50","Davi Lucas","Davi Lucas@gmail.com"), ("51","Noah","Noah@bh.com"), ("52","João Gabriel","João Gabriel@bh.com"), ("53","João Victor","João Victor@bh.com"), ("54","Luiz Miguel","Luiz Miguel@gmail.com"), ("55","Francisco","Francisco@bh.com"), ("56","Kaique","Kaique@yahoo.com"), ("57","Otávio","Otávio@gmail.com"), ("58","Augusto","Augusto@gmail.com"), ("59","Levi","Levi@yahoo.com"), ("60","Yuri","Yuri@yahoo.com"), ("61","Enrico","Enrico@yahoo.com"), ("62","Thiago","Thiago@yahoo.com"), ("63","Ian","Ian@gmail.com"), ("64","Victor Hugo","Victor Hugo@yahoo.com"), ("65","Thomas","Thomas@yahoo.com"), ("66","Henry","Henry@gmail.com"), ("67","Luiz Felipe","Luiz Felipe@gmail.com"), ("68","Ryan","Ryan@bh.com"), ("69","Arthur Miguel","Arthur Miguel@bh.com"), ("70","Davi Luiz","Davi Luiz@bh.com"), ("71","Nathan","Nathan@gmail.com"), ("72","Pedro Lucas","Pedro Lucas@yahoo.com"), ("73","Davi Miguel","Davi Miguel@yahoo.com"), ("74","Raul","Raul@bh.com"), ("75","Pedro Miguel","Pedro Miguel@gmail.com"), ("76","Luiz Henrique","Luiz Henrique@bh.com"), ("77","Luan","Luan@gmail.com"), ("78","Erick","Erick@yahoo.com"), ("79","Martin","Martin@yahoo.com"), ("80","Bruno","Bruno@yahoo.com"), ("81","Rodrigo","Rodrigo@gmail.com"), ("82","Luiz Gustavo","Luiz Gustavo@yahoo.com"), ("83","Arthur Gabriel","Arthur Gabriel@yahoo.com"), ("84","Breno","Breno@yahoo.com"), ("85","Kauê","Kauê@yahoo.com"), ("86","Enzo Miguel","Enzo Miguel@gmail.com"), ("87","Fernando","Fernando@gmail.com"), ("88","Arthur Henrique","Arthur Henrique@yahoo.com"), ("89","Luiz Otávio","Luiz Otávio@gmail.com"), ("90","Carlos Eduardo","Carlos Eduardo@gmail.com");


-- Descarta tabela Sobrenome
DROP TABLE Sobrenome;

--Gera Tabela Sobrenome
CREATE TABLE Sobrenome (
  ID_Cliente mediumint(8) unsigned NOT NULL auto_increment,
  Sobrenome varchar(255) default NULL
  PRIMARY KEY (ID_Cliente)
) AUTO_INCREMENT=1;

-- Popula Tabela Sobrenome
INSERT INTO Cliente (ID_Cliente, Nome, Email) VALUES ("1","Miguel","Miguel@bh.com"), ("2","Arthur","Arthur@gmail.com"), ("3","Bernardo","Bernardo@bh.com"), ("4","Heitor","Heitor@bh.com"), ("5","Davi","Davi@yahoo.com"), ("6","Lorenzo","Lorenzo@yahoo.com"), ("7","Théo","Théo@yahoo.com"), ("8","Pedro","Pedro@yahoo.com"), ("9","Gabriel","Gabriel@yahoo.com"), ("10","Enzo","Enzo@gmail.com"), ("11","Matheus","Matheus@gmail.com"), ("12","Lucas","Lucas@gmail.com"), ("13","Benjamin","Benjamin@yahoo.com"), ("14","Nicolas","Nicolas@gmail.com"), ("15","Guilherme","Guilherme@yahoo.com"), ("16","Rafael","Rafael@gmail.com"), ("17","Joaquim","Joaquim@gmail.com"), ("18","Samuel","Samuel@yahoo.com"), ("19","Enzo Gabriel","Enzo Gabriel@gmail.com"), ("20","João Miguel","João Miguel@gmail.com"), ("21","Henrique","Henrique@yahoo.com"), ("22","Gustavo","Gustavo@gmail.com"), ("23","Murilo","Murilo@gmail.com"), ("24","Pedro Henrique","Pedro Henrique@bh.com"), ("25","Pietro","Pietro@gmail.com"), ("26","Lucca","Lucca@gmail.com"), ("27","Felipe","Felipe@yahoo.com"), ("28","João Pedro","João Pedro@gmail.com"), ("29","Isaac","Isaac@yahoo.com"), ("30","Benício","Benício@yahoo.com"), ("31","Daniel","Daniel@gmail.com"), ("32","Anthony","Anthony@gmail.com"), ("33","Leonardo","Leonardo@gmail.com"), ("34","Davi Lucca","Davi Lucca@yahoo.com"), ("35","Bryan","Bryan@bh.com"), ("36","Eduardo","Eduardo@bh.com"), ("37","João Lucas","João Lucas@yahoo.com"), ("38","Victor","Victor@bh.com"), ("39","João","João@yahoo.com"), ("40","Cauã","Cauã@bh.com"), ("41","Antônio","Antônio@yahoo.com"), ("42","Vicente","Vicente@yahoo.com"), ("43","Caleb","Caleb@bh.com"), ("44","Gael","Gael@yahoo.com"), ("45","Bento","Bento@gmail.com"), ("46","Caio","Caio@bh.com"), ("47","Emanuel","Emanuel@yahoo.com"), ("48","Vinícius","Vinícius@bh.com"), ("49","João Guilherme","João Guilherme@bh.com"), ("50","Davi Lucas","Davi Lucas@gmail.com"), ("51","Noah","Noah@bh.com"), ("52","João Gabriel","João Gabriel@bh.com"), ("53","João Victor","João Victor@bh.com"), ("54","Luiz Miguel","Luiz Miguel@gmail.com"), ("55","Francisco","Francisco@bh.com"), ("56","Kaique","Kaique@yahoo.com"), ("57","Otávio","Otávio@gmail.com"), ("58","Augusto","Augusto@gmail.com"), ("59","Levi","Levi@yahoo.com"), ("60","Yuri","Yuri@yahoo.com"), ("61","Enrico","Enrico@yahoo.com"), ("62","Thiago","Thiago@yahoo.com"), ("63","Ian","Ian@gmail.com"), ("64","Victor Hugo","Victor Hugo@yahoo.com"), ("65","Thomas","Thomas@yahoo.com"), ("66","Henry","Henry@gmail.com"), ("67","Luiz Felipe","Luiz Felipe@gmail.com"), ("68","Ryan","Ryan@bh.com"), ("69","Arthur Miguel","Arthur Miguel@bh.com"), ("70","Davi Luiz","Davi Luiz@bh.com"), ("71","Nathan","Nathan@gmail.com"), ("72","Pedro Lucas","Pedro Lucas@yahoo.com"), ("73","Davi Miguel","Davi Miguel@yahoo.com"), ("74","Raul","Raul@bh.com"), ("75","Pedro Miguel","Pedro Miguel@gmail.com"), ("76","Luiz Henrique","Luiz Henrique@bh.com"), ("77","Luan","Luan@gmail.com"), ("78","Erick","Erick@yahoo.com"), ("79","Martin","Martin@yahoo.com"), ("80","Bruno","Bruno@yahoo.com"), ("81","Rodrigo","Rodrigo@gmail.com"), ("82","Luiz Gustavo","Luiz Gustavo@yahoo.com"), ("83","Arthur Gabriel","Arthur Gabriel@yahoo.com"), ("84","Breno","Breno@yahoo.com"), ("85","Kauê","Kauê@yahoo.com"), ("86","Enzo Miguel","Enzo Miguel@gmail.com"), ("87","Fernando","Fernando@gmail.com"), ("88","Arthur Henrique","Arthur Henrique@yahoo.com"), ("89","Luiz Otávio","Luiz Otávio@gmail.com"), ("90","Carlos Eduardo","Carlos Eduardo@gmail.com");



-- Descarta Tabela Fornecedor
DROP TABLE Fornecedor;

-- Gera Tabela Fornecedor
CREATE TABLE Fornecedor (
  ID_Fornecedor  mediumint(8) unsigned NOT NULL auto_increment,
  Nome varchar(255) default NULL,
  Email varchar(255) default NULL,
  PRIMARY KEY (ID_Fornecedor)
) AUTO_INCREMENT=1;

-- Popula Tabela Fornecedor
INSERT INTO Fornecedor (ID_Fornecedor,Nome,Email) VALUES (" 1", "Tomás", "Tomás@gmail.com"), (" 2", "Lucas Gabriel", "Lucas Gabriel@gmail.com"), (" 3", "André", "André@gmail.com"), (" 4", "José", "José@yahoo.com"), (" 5", "Yago", "Yago@bh.com"), (" 6", "Danilo", "Danilo@yahoo.com"), (" 7", "Anthony Gabriel", "Anthony Gabriel@bh.com"), (" 8", "Ruan", "Ruan@bh.com"), (" 9", "Miguel Henrique", "Miguel Henrique@yahoo.com"), ("10", "Oliver", "Oliver@yahoo.com");

-- Descarta Tabela Ordem
DROP TABLE Ordem;

-- Gera Tabela ordem
CREATE TABLE Ordem (
  ID_Ordem  mediumint(8) unsigned NOT NULL auto_increment,
  ID_Cliente mediumint,
  ID_Produto mediumint,
  Data datetime,
  PRIMARY KEY (ID_Ordem)
) AUTO_INCREMENT=1;

-- Popula Tabela Ordem
INSERT INTO Ordem (ID_Ordem, ID_Cliente, ID_Produto, Data) VALUES ("  1", "77", "19", "2019-04-22"), ("  2", "89", " 5", "2019-04-21"), ("  3", "52", "32", "2019-03-10"), ("  4", "25", "75", "2019-09-29"), ("  5", "63", "50", "2019-01-08"), ("  6", "53", "29", "2019-04-25"), ("  7", "87", "17", "2019-04-23"), ("  8", "30", "21", "2019-10-12"), ("  9", "35", "38", "2019-06-04"), (" 10", "85", "55", "2019-04-21"), (" 11", "17", "87", "2019-01-07"), (" 12", "46", "27", "2019-09-22"), (" 13", "23", "48", "2019-08-13"), (" 14", "80", "71", "2019-01-03"), (" 15", "33", "40", "2019-07-15"), (" 16", "12", "82", "2019-03-20"), (" 17", "30", "91", "2019-12-22"), (" 18", "45", "41", "2019-03-15"), (" 19", "78", "61", "2019-01-09"), (" 20", "46", "60", "2019-02-08"), (" 21", "74", "95", "2019-02-27"), (" 22", "16", "79", "2019-03-06"), (" 23", "69", "29", "2019-08-19"), (" 24", "77", "71", "2019-10-04"), (" 25", " 9", "58", "2019-04-07"), (" 26", "10", "72", "2019-05-08"), (" 27", "14", "92", "2019-06-25"), (" 28", "15", "75", "2019-03-10"), (" 29", "55", " 2", "2019-03-08"), (" 30", "78", "56", "2019-02-21"), (" 31", "72", "84", "2019-07-06"), (" 32", "68", "20", "2019-06-26"), (" 33", "61", "86", "2019-06-27"), (" 34", "78", "90", "2019-01-04"), (" 35", "86", "32", "2019-05-23"), (" 36", "90", "93", "2019-06-24"), (" 37", "31", "82", "2019-10-31"), (" 38", "88", "17", "2019-12-05"), (" 39", "86", "79", "2019-10-22"), (" 40", "15", "46", "2019-03-22"), (" 41", "79", " 9", "2019-02-07"), (" 42", "43", "61", "2019-06-27"), (" 43", "33", "60", "2019-04-16"), (" 44", "16", "87", "2019-07-18"), (" 45", "20", "76", "2019-01-23"), (" 46", "86", " 5", "2019-02-08"), (" 47", "60", "79", "2019-01-22"), (" 48", "54", "21", "2019-08-16"), (" 49", "84", "17", "2019-11-01"), (" 50", "90", "53", "2019-11-14"), (" 51", "50", "15", "2019-03-17"), (" 52", "72", "85", "2019-10-23"), (" 53", "44", "69", "2019-05-30"), (" 54", "29", "68", "2019-07-05"), (" 55", "68", "20", "2019-02-03"), (" 56", "88", "79", "2019-03-23"), (" 57", "12", "96", "2019-08-09"), (" 58", "73", "53", "2019-04-25"), (" 59", " 1", "64", "2019-07-29"), (" 60", " 7", "81", "2019-12-11"), (" 61", "46", "25", "2019-04-24"), (" 62", "24", " 1", "2019-06-05"), (" 63", "49", "86", "2019-09-11"), (" 64", "68", " 9", "2019-12-19"), (" 65", "25", " 7", "2019-12-02"), (" 66", " 6", "94", "2019-07-27"), (" 67", "76", " 8", "2019-12-29"), (" 68", "13", "85", "2019-10-09"), (" 69", "62", "71", "2019-12-13"), (" 70", "49", "49", "2019-09-03"), (" 71", "75", " 3", "2019-01-04"), (" 72", "49", "31", "2019-01-27"), (" 73", "82", "67", "2019-07-06"), (" 74", "62", "55", "2019-01-10"), (" 75", " 1", "27", "2019-09-20"), (" 76", "64", "61", "2019-12-01"), (" 77", "72", "19", "2019-08-15"), (" 78", "25", "16", "2019-02-07"), (" 79", " 8", " 5", "2019-03-18"), (" 80", "63", " 6", "2019-05-15"), (" 81", "20", "21", "2019-09-26"), (" 82", " 6", "94", "2019-03-24"), (" 83", "20", "16", "2019-02-04"), (" 84", "73", "28", "2019-06-10"), (" 85", "44", "11", "2019-10-21"), (" 86", "40", "69", "2019-12-12"), (" 87", " 2", "34", "2019-11-18"), (" 88", "73", "30", "2019-02-26"), (" 89", "62", "34", "2019-04-27"), (" 90", "59", "51", "2019-01-20"), (" 91", "61", "14", "2019-03-21"), (" 92", "86", "85", "2019-08-18"), (" 93", "42", "44", "2019-11-21"), (" 94", "76", "40", "2019-09-04"), (" 95", "27", "52", "2019-08-06"), (" 96", "29", "74", "2019-06-12"), (" 97", "19", " 4", "2019-08-21"), (" 98", " 3", "87", "2019-08-02"), (" 99", " 1", "91", "2019-01-05"), ("100", "37", "55", "2019-02-20");

-- Descarta Tabela Produto
DROP TABLE Produto;

-- Gera Tabela Produto
CREATE TABLE Produto (
  ID_Produto mediumint(8) unsigned NOT NULL auto_increment,
  ID_Fornecedor mediumint,
  Nome varchar(255) default NULL,
  PRIMARY KEY (ID_Produto)
) AUTO_INCREMENT=1;

-- Popula Tabela Produto
INSERT INTO Produto (ID_Produto, ID_Fornecedor, Nome) VALUES ('  1', ' 4', 'Alice'), ('  2', ' 5', 'Sophia'), ('  3', ' 4', 'Helena'), ('  4', ' 6', 'Valentina'), ('  5', ' 9', 'Laura'), ('  6', '10', 'Isabella'), ('  7', ' 1', 'Manuela'), ('  8', ' 1', 'Júlia'), ('  9', ' 2', 'Heloísa'), (' 10', ' 6', 'Luiza'), (' 11', ' 2', 'Maria Luiza'), (' 12', ' 9', 'Lorena'), (' 13', ' 4', 'Lívia'), (' 14', ' 2', 'Giovanna'), (' 15', ' 7', 'Maria Eduarda'), (' 16', ' 6', 'Beatriz'), (' 17', ' 7', 'Maria Clara'), (' 18', ' 9', 'Cecília'), (' 19', ' 8', 'Eloá'), (' 20', ' 6', 'Lara'), (' 21', ' 8', 'Maria Júlia'), (' 22', ' 7', 'Isadora'), (' 23', ' 3', 'Mariana'), (' 24', '10', 'Emanuelly'), (' 25', ' 1', 'Ana Júlia'), (' 26', ' 7', 'Ana Luiza'), (' 27', '10', 'Ana Clara'), (' 28', '10', 'Melissa'), (' 29', '10', 'Yasmin'), (' 30', ' 6', 'Maria Alice'), (' 31', ' 5', 'Isabelly'), (' 32', ' 3', 'Lavínia'), (' 33', ' 9', 'Esther'), (' 34', ' 6', 'Sarah'), (' 35', ' 5', 'Elisa'), (' 36', ' 2', 'Antonella'), (' 37', ' 5', 'Rafaela'), (' 38', ' 1', 'Maria Cecília'), (' 39', '10', 'Liz'), (' 40', ' 8', 'Marina'), (' 41', ' 6', 'Nicole'), (' 42', ' 1', 'Maitê'), (' 43', ' 5', 'Isis'), (' 44', ' 6', 'Alícia'), (' 45', ' 5', 'Luna'), (' 46', ' 5', 'Rebeca'), (' 47', '10', 'Agatha'), (' 48', ' 9', 'Letícia'), (' 49', '10', 'Maria-'), (' 50', ' 8', 'Gabriela'), (' 51', ' 2', 'Ana Laura'), (' 52', ' 2', 'Catarina'), (' 53', ' 6', 'Clara'), (' 54', ' 8', 'Ana Beatriz'), (' 55', ' 6', 'Vitória'), (' 56', ' 7', 'Olívia'), (' 57', '10', 'Maria Fernanda'), (' 58', '10', 'Emilly'), (' 59', ' 7', 'Maria Valentina'), (' 60', ' 7', 'Milena'), (' 61', ' 9', 'Maria Helena'), (' 62', '10', 'Bianca'), (' 63', ' 3', 'Larissa'), (' 64', ' 8', 'Mirella'), (' 65', ' 1', 'Maria Flor'), (' 66', ' 2', 'Allana'), (' 67', ' 2', 'Ana Sophia'), (' 68', ' 3', 'Clarice'), (' 69', ' 9', 'Pietra'), (' 70', ' 3', 'Maria Vitória'), (' 71', ' 6', 'Maya'), (' 72', ' 5', 'Laís'), (' 73', ' 3', 'Ayla'), (' 74', '10', 'Ana Lívia'), (' 75', ' 6', 'Eduarda'), (' 76', ' 6', 'Mariah'), (' 77', ' 6', 'Stella'), (' 78', ' 2', 'Ana'), (' 79', ' 4', 'Gabrielly'), (' 80', ' 2', 'Sophie'), (' 81', ' 4', 'Carolina'), (' 82', ' 5', 'Maria Laura'), (' 83', ' 5', 'Maria Heloísa'), (' 84', ' 5', 'Maria Sophia'), (' 85', ' 8', 'Fernanda'), (' 86', ' 6', 'Malu'), (' 87', ' 5', 'Analu'), (' 88', ' 3', 'Amanda'), (' 89', ' 7', 'Aurora'), (' 90', ' 2', 'Maria Isis'), (' 91', ' 5', 'Louise'), (' 92', ' 3', 'Heloise'), (' 93', ' 1', 'Ana Vitória'), (' 94', ' 6', 'Ana Cecília'), (' 95', ' 2', 'Ana Liz'), (' 96', ' 6', 'Joana'), (' 97', ' 6', 'Luana'), (' 98', ' 5', 'Antônia'), (' 99', ' 7', 'Isabel'), ('100', ' 9', 'Bruna');

-- Descarta Tabela Entrega
DROP TABLE Entrega;

-- Gera Tabela de Entrega
CREATE TABLE Entrega (
  ID_Entrega mediumint(8) unsigned NOT NULL auto_increment,
  ID_Produto mediumint,
  Data datetime,
  PRIMARY KEY (ID_Entrega)
) AUTO_INCREMENT=1;

-- Popula Tabela de Entrega
INSERT INTO Entrega (ID_Entrega, ID_Produto, Data) VALUES ('  1', '19', '2019-04-24'), ('  2', ' 5', '2019-04-22'), ('  3', '32', '2019-03-15'), ('  4', '75', '2019-10-01'), ('  5', '50', '2019-01-11'), ('  6', '29', '2019-04-26'), ('  7', '17', '2019-04-24'), ('  8', '21', '2019-10-15'), ('  9', '38', '2019-06-05'), (' 10', '55', '2019-04-27'), (' 11', '87', '2019-01-13'), (' 12', '27', '2019-09-28'), (' 13', '48', '2019-08-20'), (' 14', '71', '2019-01-05'), (' 15', '40', '2019-07-19'), (' 16', '82', '2019-03-23'), (' 17', '91', '2019-12-24'), (' 18', '41', '2019-03-18'), (' 19', '61', '2019-01-16'), (' 20', '60', '2019-02-13'), (' 21', '95', '2019-02-28'), (' 22', '79', '2019-03-10'), (' 23', '29', '2019-08-25'), (' 24', '71', '2019-10-09'), (' 25', '58', '2019-04-11'), (' 26', '72', '2019-05-13'), (' 27', '92', '2019-06-29'), (' 28', '75', '2019-03-16'), (' 29', ' 2', '2019-03-09'), (' 30', '56', '2019-02-28'), (' 31', '84', '2019-07-09'), (' 32', '20', '2019-06-30'), (' 33', '86', '2019-06-29'), (' 34', '90', '2019-01-08'), (' 35', '32', '2019-05-30'), (' 36', '93', '2019-06-27'), (' 37', '82', '2019-11-04'), (' 38', '17', '2019-12-06'), (' 39', '79', '2019-10-24'), (' 40', '46', '2019-03-29'), (' 41', ' 9', '2019-02-13'), (' 42', '61', '2019-06-29'), (' 43', '60', '2019-04-21'), (' 44', '87', '2019-07-24'), (' 45', '76', '2019-01-29'), (' 46', ' 5', '2019-02-11'), (' 47', '79', '2019-01-27'), (' 48', '21', '2019-08-19'), (' 49', '17', '2019-11-04'), (' 50', '53', '2019-11-21'), (' 51', '15', '2019-03-18'), (' 52', '85', '2019-10-25'), (' 53', '69', '2019-06-01'), (' 54', '68', '2019-07-10'), (' 55', '20', '2019-02-04'), (' 56', '79', '2019-03-29'), (' 57', '96', '2019-08-10'), (' 58', '53', '2019-05-01'), (' 59', '64', '2019-08-02'), (' 60', '81', '2019-12-17'), (' 61', '25', '2019-04-27'), (' 62', ' 1', '2019-06-07'), (' 63', '86', '2019-09-12'), (' 64', ' 9', '2019-12-26'), (' 65', ' 7', '2019-12-09'), (' 66', '94', '2019-07-29'), (' 67', ' 8', '2019-12-31'), (' 68', '85', '2019-10-14'), (' 69', '71', '2019-12-19'), (' 70', '49', '2019-09-04'), (' 71', ' 3', '2019-01-07'), (' 72', '31', '2019-02-01'), (' 73', '67', '2019-07-07'), (' 74', '55', '2019-01-17'), (' 75', '27', '2019-09-24'), (' 76', '61', '2019-12-05'), (' 77', '19', '2019-08-19'), (' 78', '16', '2019-02-13'), (' 79', ' 5', '2019-03-20'), (' 80', ' 6', '2019-05-17'), (' 81', '21', '2019-10-01'), (' 82', '94', '2019-03-25'), (' 83', '16', '2019-02-06'), (' 84', '28', '2019-06-17'), (' 85', '11', '2019-10-28'), (' 86', '69', '2019-12-13'), (' 87', '34', '2019-11-19'), (' 88', '30', '2019-02-27'), (' 89', '34', '2019-05-02'), (' 90', '51', '2019-01-27'), (' 91', '14', '2019-03-24'), (' 92', '85', '2019-08-20'), (' 93', '44', '2019-11-24'), (' 94', '40', '2019-09-05'), (' 95', '52', '2019-08-09'), (' 96', '74', '2019-06-14'), (' 97', ' 4', '2019-08-24'), (' 98', '87', '2019-08-08'), (' 99', '91', '2019-01-07'), ('100', '55', '2019-02-27');


-- 1) π_(IDCliente,   Nome,   Email) (Cliente)
SELECT *
FROM Cliente 
LIMIT 10;

-- 2) π_(IDProduto,   IDFornecedor,   Nome) (Produto)
SELECT * 
FROM Produto
LIMIT 10;

-- 3) Cliente  ⋈_(IDCliente=IDCliente)  Ordem
SELECT c.ID_Cliente, c.Nome, c.Email, o.ID_Ordem, o.ID_Produto, o.Data 
FROM Cliente as c 
JOIN Ordem as o ON c.ID_Cliente = o.ID_Cliente
LIMIT 10; 

-- 4) Produto  ⋈_(IDProduto=IDProduto)  Ordem
SELECT p.ID_Produto, p.ID_Fornecedor, p.Nome as Nome_Produto, o.ID_Ordem, o.ID_Cliente, o.Data 
FROM Produto as p 
JOIN Ordem as o ON p.ID_Produto = o.ID_Produto
LIMIT 10;

-- 5) Fornecedor  ⋈_(IDFornecedor=IDFornecedor)  Produto
SELECT f.ID_Fornecedor, f.Nome, f.Email, p.ID_Produto, p.Nome as Nome_Produto 
FROM Fornecedor as f 
JOIN Produto as p ON f.ID_Fornecedor = p.ID_Fornecedor
LIMIT 10;

-- 6) (Fornecedor  ⋈_(IDFornecedor=IDFornecedor)  Produto)⋈_(IDProduto=IDProduto)  Ordem
SELECT f.ID_Fornecedor, f.Nome, f.Email, p.ID_Produto, p.Nome as Nome_Produto, o.ID_Cliente, o.Data 
FROM Fornecedor as f 
JOIN Produto as p ON f.ID_Fornecedor = p.ID_Fornecedor 
JOIN Ordem as o ON p.ID_Produto = o.ID_Produto
LIMIT 10;

-- 7) (Produto  ⋈_(IDProduto=IDProduto)  Ordem)  ⋈_(IDCliente=IDCliente)  Cliente
SELECT p.ID_Produto, p.ID_Fornecedor, p.Nome as Nome_Produto, o.ID_Ordem, o.ID_Cliente, o.Data 
FROM Produto as p 
JOIN Ordem as o ON p.ID_Produto = o.ID_Produto
JOIN Cliente as c ON o.ID_Cliente = c.ID_Cliente
LIMIT 10;

-- 8) (Produto  ⋈_(IDProduto=IDProduto)  Ordem)  ⋈_(IDFornecedor=IDFornecedor)  Fornecedor
SELECT p.ID_Produto, p.ID_Fornecedor, p.Nome as Nome_Produto, o.ID_Ordem, o.ID_Cliente, o.Data 
FROM Produto as p 
JOIN Ordem as o ON p.ID_Produto = o.ID_Produto
JOIN Fornecedor as f ON p.ID_Fornecedor = f.ID_Fornecedor
LIMIT 10;

-- 9) J_(COUNT IDCliente) (Cliente  ⋈_(IDCliente=IDCliente)  Ordem) 
SELECT c.ID_Cliente, COUNT(o.ID_Ordem) as Frequencia 
FROM Cliente as c 
JOIN Ordem as o ON c.ID_Cliente = o.ID_Cliente
GROUP BY c.ID_Cliente
ORDER BY Frequencia DESC
LIMIT 10; 

-- 10) J_(COUNT IDProduto) (Fornecedor  ⋈_(IDFornecedor=IDFornecedor)  Produto) 
SELECT f.ID_Fornecedor, COUNT(p.ID_Produto) as Frequencia
FROM Fornecedor as f
JOIN Produto as p ON f.ID_Fornecedor = p.ID_Fornecedor
GROUP BY f.ID_Fornecedor
ORDER BY Frequencia DESC
LIMIT 10; 


In [ ]:
%timeit pd.read_sql('SELECT * FROM Ordem', con = engine) 


In [ ]:
#@title Declara tabelas (dataframes)

# Declara data frame de Clientes com 100 clientes
Cliente = pd.DataFrame({'ID_Cliente' : range(89),
                       'Nome' : Homens[0:89],
                       'Email' : email_Cliente})

# Imprime Cliente
print(Cliente.head())

# Declara data frame de Fornecedor com 100 Fornecedores
Fornecedor = pd.DataFrame({'ID_Fornecedor' : range(9),
                           'Nome' : Homens[90:99],
                           'Email' : email_Fornecedor})

# Imprime Fornecedor
print(Fornecedor.head())

# Declara data frame de Ordem com 500 ordens
Ordem = pd.DataFrame({'ID_Ordem' : range(5000),
                      'ID_Cliente' : np.random.choice(range(89), 5000).tolist(),
                      'ID_Produto' : np.random.choice(range(100), 5000).tolist(),
                      'Data' : Data_Ordem})

# Remove Horario
Ordem['Data'] = [str(i) for i in pd.to_datetime(Ordem['Data']).dt.date]

# Imprime Ordem
print(Ordem.head())

# Declara dataframe de Produto com 100 produtos
Produto = pd.DataFrame({'ID_Produto' : range(100),
                        'ID_Fornecedor' : np.random.choice(range(10), 100).tolist(),
                        'Nome' : Mulheres[0:100] })

# Imprime Produto
print(Produto.head())

# Declara data frame de entrega com 100 entregas
Entrega = pd.DataFrame({'ID_Entrega' : range(5000),
                        'ID_Produto' : Ordem.ID_Produto,
                        'Data' : Data_Entrega})

# Remove Horario
Entrega['Data'] = [str(i) for i in pd.to_datetime(Entrega['Data']).dt.date]

# Imprime Entrega
print(Entrega.head())

# Declara Data Frame Sobrenome
Sobrenome = pd.DataFrame({'ID_Cliente' : range(90),
                          'Sobrenome' : Sobrenomes})

# Imprime Sobrenome
print(Sobrenome.head())
